In [1]:
import requests
import csv
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from sources import api
import time
import random


####this script needs a  watchlist.csv file containing crypto & crypto IDs on each line, separated by commas, e.g.:
#BTC,1
#ETH,1027

# API key from CoinMarketCap (replace with your own API key)
API_KEY = api

def fetch_historical_ohlcv_data(cryptocurrency_id, symbol, date, output_dir):
    url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical"
    headers = {
        'X-CMC_PRO_API_KEY': API_KEY
    }
    time_start = date.strftime('%Y-%m-%dT00:00:00Z')
    time_end = date.strftime('%Y-%m-%dT23:59:59Z')
    params = {
        'id': cryptocurrency_id,
        'symbol': symbol,
        'time_start': time_start,
        'time_end': time_end,
        'interval': '5m',
        'count': 288,
        'convert': 'USD,ETH,BTC'
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        json_data = response.json()['data']
        quotes = json_data['quotes']
        return quotes
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return []

def save_to_csv(data, output_dir, symbol, date):
    output_subdir = os.path.join(output_dir, symbol)
    os.makedirs(output_subdir, exist_ok=True)
    output_path = os.path.join(output_subdir, f"{date}.csv")

    with open(output_path, 'w', newline='') as csvfile:
        fieldnames = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'market_cap',
                      'open_eth', 'high_eth', 'low_eth', 'close_eth', 'volume_eth', 'market_cap_eth',
                      'open_btc', 'high_btc', 'low_btc', 'close_btc', 'volume_btc', 'market_cap_btc']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for quote in data:
            timestamp = quote['time_open']
            quote_usd = quote['quote']['USD']
            quote_eth = quote['quote']['ETH']
            quote_btc = quote['quote']['BTC']

            row = {
                'timestamp': timestamp,
                'open': quote_usd['open'],
                'high': quote_usd['high'],
                'low': quote_usd['low'],
                'close': quote_usd['close'],
                'volume': quote_usd['volume'],
                'market_cap': quote_usd['market_cap'],
                'open_eth': quote_eth['open'],
                'high_eth': quote_eth['high'],
                'low_eth': quote_eth['low'],
                'close_eth': quote_eth['close'],
                'volume_eth': quote_eth['volume'],
                'market_cap_eth': quote_eth['market_cap'],
                'open_btc': quote_btc['open'],
                'high_btc': quote_btc['high'],
                'low_btc': quote_btc['low'],
                'close_btc': quote_btc['close'],
                'volume_btc': quote_btc['volume'],
                'market_cap_btc': quote_btc['market_cap']
            }
            writer.writerow(row)

def backfill_data(ticker, cryptocurrency_id, start_date, end_date, output_dir, overwrite):
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        output_path = os.path.join(output_dir, ticker, f"{date_str}.csv")

        if not overwrite and os.path.exists(output_path):
            with open(output_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                row_count = sum(1 for row in reader) - 1  # Subtract 1 for the header row
                if row_count >= 288:
                    print(f"Skipping {ticker} - {date_str} (already exists with sufficient rows)")
                    current_date += timedelta(days=1)
                    continue

        print(f"Fetching data for {ticker} - {date_str}")
        data = fetch_historical_ohlcv_data(cryptocurrency_id, ticker, current_date, output_dir)
        save_to_csv(data, output_dir, ticker, date_str)
        current_date += timedelta(days=1)
        time.sleep(1)  # Add a delay of 1 second between requests to respect API limits

def main(watchlist_file, start_date, end_date, output_dir, overwrite):
    with open(watchlist_file, 'r') as file:
        watchlist = [line.strip().split(',') for line in file]

    with ThreadPoolExecutor() as executor:
        futures = []
        for ticker, cryptocurrency_id in watchlist:
            futures.append(executor.submit(backfill_data, ticker, cryptocurrency_id, start_date, end_date, output_dir, overwrite))

        for future in futures:
            future.result()

if __name__ == '__main__':
    # Example usage
    watchlist_file = 'watchlist.csv'
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31)
    output_dir = 'historical_data'
    overwrite = False

    main(watchlist_file, start_date, end_date, output_dir, overwrite)

Fetching data for ETH - 2022-01-01Fetching data for BTC - 2022-01-01

Error: 403
{
    "status": {
        "timestamp": "2024-03-08T22:01:18.598Z",
        "error_code": 1006,
        "error_message": "Your API Key subscription plan doesn't support this endpoint.",
        "elapsed": 0,
        "credit_count": 0
    }
}
Error: 403
{
    "status": {
        "timestamp": "2024-03-08T22:01:18.603Z",
        "error_code": 1006,
        "error_message": "Your API Key subscription plan doesn't support this endpoint.",
        "elapsed": 0,
        "credit_count": 0
    }
}
Fetching data for BTC - 2022-01-02Fetching data for ETH - 2022-01-02

Error: 403
{
    "status": {
        "timestamp": "2024-03-08T22:01:19.851Z",
        "error_code": 1006,
        "error_message": "Your API Key subscription plan doesn't support this endpoint.",
        "elapsed": 0,
        "credit_count": 0
    }
}
Error: 403
{
    "status": {
        "timestamp": "2024-03-08T22:01:20.130Z",
        "error_code": 1006